# GPU Check

In [ ]:
import GPUtil
GPUs = GPUtil.getGPUs()
for gpu in GPUs:
  print(gpu.name, gpu.memoryTotal)

# imports

In [ ]:
from search_eval.eval_MultiTrial import Eval_MT
from search_eval.optimizer.SingleImageDataset import SingleImageDataset
from search_eval.utils.common_utils import *
from search_space.search_space import DARTS_UNet

from nni import trace
import nni.retiarii.strategy as strategy

from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader

import torch
import numpy as np

torch.cuda.empty_cache()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print('CUDA available: {}'.format(torch.cuda.is_available()))

# Choose Strategy

In [ ]:
# Select a Search Strategy
# search_strategy = strategy.Random(dedup=True)
# search_strategy = strategy.TPE()
search_strategy = strategy.RegularizedEvolution(dedup=True)

# multi search

In [ ]:
total_iterations = 700

resolution = 64
noise_type = 'gaussian'
noise_level = '0.09'
phantom_num = 45
phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{phantom_num}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{phantom_num}.npy')


# Create the lightning module
module = Eval_MT(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,

                learning_rate=0.098, 
                buffer_size=700,
                patience=150,
                weight_decay=1.9e-7,
                
                report_every=10,
                HPO=True
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )

if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = trace(DataLoader)(SingleImageDataset(phantom, num_iter=1), batch_size=1)
val_loader = trace(DataLoader)(SingleImageDataset(phantom, num_iter=1), batch_size=1)
lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=None)

# Create a Search Space
model_space = DARTS_UNet()


# Configure and run the experiment for multi-strategy
experiment = RetiariiExperiment(model_space, lightning, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = f'MT_{search_strategy.__class__.__name__}_p{phantom_num}_NT{noise_type}_NL{noise_level}'

exp_config.max_trial_number = 250   # spawn 4 trials at most
exp_config.trial_concurrency = 3  # will run two trials concurrently

exp_config.trial_gpu_number = 1
exp_config.training_service.use_active_gpu = True

experiment.run(exp_config, 8889) # recommended in documentation was 8081

In [ ]:
exported_arch = experiment.export_top_models()
exported_arch

In [ ]:
experiment.stop()

# clear the cuda cache
torch.cuda.empty_cache()

# all searches

In [ ]:
# Select a Search Strategy
search_strategy = strategy.Random(dedup=True)
# search_strategy = strategy.TPE()
# search_strategy = strategy.RegularizedEvolution(dedup=True)

resolution = 64

for noise_level in ['0.05', '0.09', '0.15', '0.20']:
    # may need to incorporate some logic to scale the number of iterations based on the noise level
    total_iterations = 1200
    for noise_type in ['gaussian', 'poisson']:
        for img in range(0, 5):
            phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{img}.npy')
            phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{img}.npy')
            # Create the lightning module
            module = Eval_MT(
                            phantom=phantom, 
                            phantom_noisy=phantom_noisy,
                            lr=0.01, 
                            buffer_size=100,
                            patience=1000,
                            )

            # Create a PyTorch Lightning trainer
            trainer = Trainer(
                        max_epochs=total_iterations,
                        fast_dev_run=False,
                        gpus=1,
                        )

            if not hasattr(trainer, 'optimizer_frequencies'):
                trainer.optimizer_frequencies = []

            # Create the lighting object for evaluator
            train_loader = trace(DataLoader)(SingleImageDataset(phantom, num_iter=1), batch_size=1)
            val_loader = trace(DataLoader)(SingleImageDataset(phantom, num_iter=1), batch_size=1)
            lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=val_loader)

            # Create a Search Space
            model_space = DARTS_UNet()

            # Configure and run the experiment for multi-strategy
            experiment = RetiariiExperiment(model_space, lightning, [], search_strategy)
            exp_config = RetiariiExeConfig('local')
            exp_config.experiment_name = f'MT_{search_strategy.__class__.__name__}'

            exp_config.max_trial_number = 100   # spawn 4 trials at most
            exp_config.trial_concurrency = 1  # will run two trials concurrently

            exp_config.trial_gpu_number = 1
            exp_config.training_service.use_active_gpu = True

            experiment.run(exp_config, 8081)
            exported_arch = experiment.export_top_models()
            # write this out to a file
            # include the image, hyperparameters, the architecture and the final loss
            # this will be used to create a table of results
            # also include the number of parameters

            exported_arch

            # stop the experiment
            experiment.stop()

            # clear the cuda cache
            torch.cuda.empty_cache()